# General stuff

In [66]:
import glob
import re
import pandas as pd
import moviepy
from moviepy.editor import *
import subprocess
import shlex

MOVIES_PATH = "./data"


In [2]:
MOVIE_REGEX = '.*The[.]Simpsons[.]S(?P<season>\d\d)E(?P<episode>\d\d)[.].*'
movie_files = glob.glob(MOVIES_PATH+"/*.mkv")

movie_files_df = pd.DataFrame(
    [ [m.group(0), m.group("season"), m.group("episode")] for m in [ re.search(MOVIE_REGEX, f) for f in movie_files] ],
    columns=["filename", "season", "episode"]
)

movie_files_df.to_csv("data/movie_files.csv")

In [3]:
movie_files_df.groupby("season").count()

,filename,episode
season,,
01,1,1
02,1,1
03,1,1
06,3,3
07,1,1
08,1,1
11,22,22
14,1,1
16,1,1


# Concatenating Segments

In [93]:
TMP_DIR = "/tmp"

def create_movie(output_name, segments, movie_files_df=movie_files_df):
    tmp_dir = TMP_DIR + "/" + output_name
    # create temp dir
    subprocess.call("rm -fr " + tmp_dir, shell=True)
    subprocess.call("mkdir " + tmp_dir, shell=True)
    
    list_file = "{}/list".format(tmp_dir)
    with open(list_file, 'w') as f:
        for i, segment in enumerate(segments):
            file_id, from_sec, to_sec = segment
            print("Extracting segment #{} ({}-{} in file #{})...".format(i, from_sec, to_sec, file_id))

            input_file = movie_files_df.ix[file_id].filename
            output_file = "{}/{}.mkv".format(tmp_dir, i)
            
            ffmpeg_command = "ffmpeg -i {} -ss {} -to {} -async 1 -strict -2 {}".format(shlex.quote(input_file), from_sec, to_sec, output_file)
            f.write("file {}\n".format(output_file))

            subprocess.call(ffmpeg_command, shell=True)

    print("Concatenating all files...")
    subprocess.call("ffmpeg -safe 0 -f concat -i {} -async 1 -strict -2 {}".format(list_file, output_name), shell=True)
    print("Done. {} was created".format(output_name))

In [94]:
segments = [[0,100,102], [1,500,503], [2,700,704]]

create_movie("test.mkv", segments)

Extracting segment #0 (100-102 in file #0)...
Extracting segment #1 (500-503 in file #1)...
Extracting segment #2 (700-704 in file #2)...
Concatenating all files...
Done. test.mkv was created


# Analyzing SRTs

In [96]:
from difflib import SequenceMatcher
SequenceMatcher(None, 'the cat is going home', 'the cats is going home').ratio()

0.9767441860465116

In [97]:
srt_files = glob.glob(MOVIES_PATH+"/*.srt")
srt_files

[]